In [ ]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config = config)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

**Note** : if you encounter an error running any of the examples below consider restarting the kernel and running this cell first

# DeepSIBA example 1 : Train ensemble
In this example a deepSIBA ensemble model will be trained from scratch using the model_params and train_params dictionaries

In [1]:
model_params = {
    "max_atoms" : int(60), "num_atom_features" : int(62), "max_degree" : int(5), "num_bond_features" : int(6),
    "graph_conv_width" : [128,128,128], "conv1d_filters" : int(128), "conv1d_size" : int(29), "dropout_encoder" : 0.25,
    "conv1d_filters_dist" : [128,128], "conv1d_size_dist" : [17,1], "dropout_dist" : 0.25, "pool_size" : int(4),
    "dense_size" : [256,128,128], "l2reg" : 0.01, "dist_thresh" : 0.2, "lr" : 0.001 
}

The model_params dictionary contains the parameters to build the deepSIBA siamese GCN architecture, more specifically:
1. **max_atoms, num_atom_features, max_degree and num_bond_features** refer to the parameters needed to featurize the input chemical structures. For more information, refer to the *ESI of the deepSIBA publication*.
2. **graph_conv_width, conv1d_filters, conv1d_size, dropout_encoder** refer to the parameters of the siamese graph encoders.
3. **conv1d_filters_dist, conv1d_size_dist, dropout_dist, pool_size, dense_size, l2reg** refer to the parameters of the distance module.
4. **dist_thresh** is the distance threshold to consider 2 chemical structures similar in biological effect (needed for custom training metrics).
5. **lr** is the learning rate.

In [2]:
train_params = {
    "cell_line" : "a375", "split" : "alldata", "number_folds" : [0,1],
    "output_dir" : "C:/Users/user/Documents/deepSIBA/results/test1",
    "batch_size" : int(128), "epochs" : int(20), 
    "N_ensemble" : int(5), "nmodel_start" : int(0), "prec_threshold" : 0.2,
    "test_value_norm" : True
}

The train_params dictionary contains the parameters required to train deepSIBA:
1. **cell_line** is the cellular model of choice out of **(a375,pc3,vcap,mcf7,merged)** for which we have enough available data. The merged option refers to data merged across cell lines.
2. **split** is one of **(train_test_split,5_fold_cv_split,alldata)**. The data to train the models are available in our google drive folder, see **data/readme.md**.
3. **number_folds** is a list, if split == train_test_split the number_folds should be [0]. If the split is a 5_fold_cv_split the number_folds should be [0,1,2,3,4] in order to train the model in all splits. If you want to train a model on a specific fold, e.g. the 3rd one, the number_folds should be [2].
4. **output_dir** is the full path to the specified output directory.
5. **N_ensemble** is the number of models to train and include in the ensemble.
6. **nmodel_start** this should be set to 0 if training for the first time, but if training is halted, nmodel_start specifies the model number in the ensemble to start training from.
7. **prec_threshold** is the distance threshold to consider 2 chemical structures similar in biological effect (needed for custom training metrics).
8. **test_value_norm** is the test/val value already normalized between 0-1.

In [3]:
from deepSIBA_train import siba_trainer
example_1 = siba_trainer(train_params, model_params)

Using TensorFlow backend.
C:\Users\user\Miniconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/20
 816/1762 [============>.................] - ETA: 1:06 - loss: 0.7726 - mean_squared_error: 0.3611 - get_cindex: 0.5007 - r_square: -29.1534 - pearson_r: -4.9208e-04 - mse_similars: nan

KeyboardInterrupt: 

# DeepSIBA example 2 : Load trained ensemble and predict
In this example a trained deepSIBA ensemble model will be loaded and used to make predictions for the appropriate test set.
For each of the cell lines, trained ensembles of either 50 or 10 models for all available splits, can be found in our google drive, see **trained_models/readme.md**.

In [4]:
model_params = {
    "max_atoms" : int(60), "num_atom_features" : int(62), "max_degree" : int(5), "num_bond_features" : int(6),
    "graph_conv_width" : [128,128,128], "conv1d_filters" : int(128), "conv1d_size" : int(29), "dropout_encoder" : 0.25,
    "conv1d_filters_dist" : [128,128], "conv1d_size_dist" : [17,1], "dropout_dist" : 0.25, "pool_size" : int(4),
    "dense_size" : [256,128,128], "l2reg" : 0.01, "dist_thresh" : 0.2, "lr" : 0.001 
}

First of all the model is compiled with the parameters (model_params) as described in example 1.

In [5]:
from deepSIBA_model import siamese_model,enc_graph
siamese_net=siamese_model(model_params)
print(siamese_net.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
atom_inputs_1 (InputLayer)      (None, 60, 62)       0                                            
__________________________________________________________________________________________________
bond_inputs_1 (InputLayer)      (None, 60, 5, 6)     0                                            
__________________________________________________________________________________________________
edge_inputs_1 (InputLayer)      (None, 60, 5)        0                                            
__________________________________________________________________________________________________
atom_inputs_2 (InputLayer)      (None, 60, 62)       0                                            
__________________________________________________________________________________________________
bond_input

<img src="architecture.png">

In [6]:
test_params = {
    "cell_line" : "a375", "split" : "5_fold_cv_split", "fold_id" : int(1),
    "N_ensemble" : int(10), "prec_threshold" : 0.2,
    "name_pattern":"siam_no_augment"
}

The test_params dictionary contains the parameters required to train deepSIBA:
1. **cell_line** is the cellular model of choice out of **(a375,pc3,vcap,mcf7)** for which we have enough available data. Later a merged option will be added.
2. **split** is one of **(train_test_split,5_fold_cv_split)**. 
3. **fold_id** is an integer, if split == train_test_split the fold_id should be 0. If the split is a 5_fold_cv_split the fold_id should be 0,1,2,3 or 4 (one less than the corresponding folder's name for this fold) in order to test the model's performance in **a specific split**.
4. **N_ensemble** is the number of total already trained models and at the same time the models included in the ensembled prediction.
5. **prec_threshold** is the distance threshold to consider 2 chemical structures similar in biological effect (needed for custom training metrics).
6. **name_pattern** is the pattern of the name of files of models' saved weights. **For example** if the weights are saved in files with names such as **siam_no_augment_18.h5** the **pattern is siam_no_augment** .

**NOTE:** The saved model weights are in the subfolders of "trained_models/" and their exact position is described fully given **cell_line** and **split** parameters.

In [7]:
from deepSIBA_ensembles import siba_val_loader
df_cold=siba_val_loader(test_params, model_params,siamese_net)

Get the ensembled predictions with siba_val_loader function.
The data frame with the predictions and the corresponding CV (coefficient of variation) of each prediction is presented below

**NOTE:** All distance values have been adjusted in the range 0 to 1.

In [8]:
df_cold

,X.1,X,Var1,Var2,value,sig_id.x,pert_iname.x,quality.x,rdkit.x,pert_dose.x,...,sig_id.y,pert_iname.y,quality.y,rdkit.y,pert_dose.y,pert_time.y,iscoldx,iscoldy,mu,cv
0,10,10,BRD-K79602928,BRD-A06352508,0.401485,BRAF001_A375_6H:BRD-K79602928-003-13-2:10,metformin,1,CN(C)C(=N)NC(=N)N,10.0,...,CPC004_A375_6H:BRD-A06352508-001-02-9:10,SB-218078,1,O=C1NC(=O)c2c1c1c3ccccc3n3c1c1c2c2ccccc2n1C1CC...,10.0,6,True,False,0.384194,0.165098
1,14,14,BRD-K79602928,BRD-A09467419,0.245354,BRAF001_A375_6H:BRD-K79602928-003-13-2:10,metformin,1,CN(C)C(=N)NC(=N)N,10.0,...,CPC004_A375_6H:BRD-A09467419-003-14-1:10,mebeverine,1,CCN(CCCCOC(=O)c1ccc(OC)c(OC)c1)C(C)Cc1ccc(OC)cc1,10.0,6,True,False,0.336458,0.159747
2,19,19,BRD-K79602928,BRD-A25687296,0.323027,BRAF001_A375_6H:BRD-K79602928-003-13-2:10,metformin,1,CN(C)C(=N)NC(=N)N,10.0,...,CPC004_A375_6H:BRD-A25687296-300-03-5:10,emetine,1,CC[C@H]1CN2CCc3cc(OC)c(OC)cc3C2C[C@@H]1CC1NCCc...,10.0,6,True,False,0.380275,0.173504
3,25,25,BRD-K79602928,BRD-A26384407,0.345075,BRAF001_A375_6H:BRD-K79602928-003-13-2:10,metformin,1,CN(C)C(=N)NC(=N)N,10.0,...,CPC004_A375_6H:BRD-A26384407-001-15-2:10,chlorthalidone,1,NS(=O)(=O)c1cc(C2(O)NC(=O)c3ccccc32)ccc1Cl,10.0,6,True,False,0.344534,0.160365
4,32,32,BRD-K79602928,BRD-A27554692,0.301334,BRAF001_A375_6H:BRD-K79602928-003-13-2:10,metformin,1,CN(C)C(=N)NC(=N)N,10.0,...,CPC004_A375_6H:BRD-A27554692-001-01-3:10,altrenogest,1,C=CC[C@]1(O)CCC2C3CCC4=CC(=O)CCC4=C3C=C[C@@]21C,10.0,6,True,False,0.326400,0.154341
5,40,40,BRD-K79602928,BRD-A29426959,0.216522,BRAF001_A375_6H:BRD-K79602928-003-13-2:10,metformin,1,CN(C)C(=N)NC(=N)N,10.0,...,CPC004_A375_6H:BRD-A29426959-050-07-4:10,carbinoxamine,1,CN(C)CCOC(c1ccc(Cl)cc1)c1ccccn1,10.0,6,True,False,0.335147,0.161825
6,49,49,BRD-K79602928,BRD-A33447119,0.296208,BRAF001_A375_6H:BRD-K79602928-003-13-2:10,metformin,1,CN(C)C(=N)NC(=N)N,10.0,...,CPC004_A375_6H:BRD-A33447119-001-03-3:10,oxfendazole,1,COC(=O)Nc1nc2ccc(S(=O)c3ccccc3)cc2[nH]1,10.0,6,True,False,0.334990,0.164446
7,59,59,BRD-K79602928,BRD-A35588707,0.260832,BRAF001_A375_6H:BRD-K79602928-003-13-2:10,metformin,1,CN(C)C(=N)NC(=N)N,10.0,...,CPC004_A375_6H:BRD-A35588707-001-03-0:10,teniposide,1,COc1cc([C@@H]2c3cc4c(cc3C(O[C@@H]3O[C@@H]5CO[C...,10.0,6,True,False,0.345862,0.156118
8,70,82,BRD-K79602928,BRD-A54880345,0.345033,BRAF001_A375_6H:BRD-K79602928-003-13-2:10,metformin,1,CN(C)C(=N)NC(=N)N,10.0,...,CPC004_A375_6H:BRD-A54880345-001-11-8:10,etomidate,1,CCOC(=O)c1cncn1C(C)c1ccccc1,10.0,6,True,False,0.346034,0.162046
9,109,124,BRD-K79602928,BRD-A93236127,0.364504,BRAF001_A375_6H:BRD-K79602928-003-13-2:10,metformin,1,CN(C)C(=N)NC(=N)N,10.0,...,CPC004_A375_6H:BRD-A93236127-001-03-7:10,digitoxin,1,C[C@H]1O[C@@H](O[C@H]2[C@@H](O)C[C@H](O[C@H]3[...,10.0,6,True,False,0.350089,0.160401


Finally the model's performance together with a scatterplot of the predicted values VS the true values of the test set, are given bellow:

**NOTE:** If there are no predictions lower than the precision threshold defined, the MSE in the predictions lower than the defined threshold and the precision will be **None**

In [9]:
from utility.evaluator import model_evaluate
import numpy as np
from matplotlib import pyplot as plt
true = np.array(df_cold.value)
pred = np.array(df_cold.mu)
get_eval=model_evaluate(pred,true,test_params["prec_threshold"],df_cold)
print(get_eval)
plt.scatter(pred,true,s = 0.5, label = "pearson`s r: "+str(round(get_eval['cor'][0],2)))
plt.xlabel("predicted GO distance")
plt.ylabel("True GO distance")
plt.legend(loc='upper left')
plt.show()

        cor  mse_all  mse_similars  precision  accuracy    recall  positives
0  0.576997  0.00841      0.004198   0.891473  0.828167  0.013958        129


<Figure size 640x480 with 1 Axes>

# DeepSIBA example 3 : Screening

In this example a trained deepSIBA ensemble model will be used to identify chemical structures from the Chembl and the CMap datasets that affect similar biological processes to a query structure. 
Given a **query chemical structure and a cellular model**:

1. If a structural analogue to the query (ECFP4 similarity > 0.9) is present in the cell line's training set, the **CMap** and the **Chembl** datasets will be screened for chemical structures that affect similar BPs to the query.
2. If no structural analogue exists, the appropriate training set from the **CMap** dataset will be screened.

**NOTE** : Our trained deepSIBA ensembles allow query structures of up to 60 atoms. 

In [10]:
model_params = {
    "max_atoms" : int(60), "num_atom_features" : int(62), "max_degree" : int(5), "num_bond_features" : int(6),
    "graph_conv_width" : [128,128,128], "conv1d_filters" : int(128), "conv1d_size" : int(29), "dropout_encoder" : 0.25,
    "conv1d_filters_dist" : [128,128], "conv1d_size_dist" : [17,1], "dropout_dist" : 0.25, "pool_size" : int(4),
    "dense_size" : [256,128,128], "l2reg" : 0.01, "dist_thresh" : 0.2, "lr" : 0.001 
}

In [11]:
screening_params = {
    "query_smile" : "CC(C)(C)c1nc(-c2cccc(NS(=O)(=O)c3c(F)cccc3F)c2F)c(-c2ccnc(N)n2)s1", 
    "cell_line" : "a375", "split" : "5_fold_cv_split" ,"database" : ["Chembl","CMap"],
    "output_dir" : "C:/Users/user/Documents/deepSIBA/results/screening_test1" , "model_path" : "", 
    "atom_limit" : int(60), "N_models" : int(10),
    "name_pattern" : "siam_no_augment", "fold_id" : int(0),
    "screening_threshold" : 0.22
}

The screening_params dictionary contains the parameters required to screen a database with deepSIBA:

1. **query_smile** is the smile string of the chemical structure.
2. **cell_line** is the cellular model of choice out of **(a375,pc3,vcap,mcf7)** for which we have enough available data. Later a merged option will be added.
3. **split** is one of **(train_test_split,5_fold_cv_split,alldata,custom)**. The split selected defines the trained model ensemble that will be loaded. For the screening application the **alldata** split is suggested, where models are trained on the entirety of available data. If **custom** is selected the user must provide a path in **model_path** to load the custom trained models (up to models/ directory).
3. **database** is a list of the supported screening databases. So far only **Chembl** and **CMap** are supported.
4. **output_dir** full path to the desired output directory to write results. The Chembl screening is performed in parts due to the size of the database.
5. **atom_limit** the specified model_params of the trained models, when the split is not **custom** these should be 60.
6. **N_ensemble** is the number of total already trained models and at the same time the models included in the ensembled prediction.
7. **name_pattern** is the pattern of the name of files of models' saved weights. **For example** if the weights are saved in files with names such as **siam_no_augment_18.h5** the **pattern is siam_no_augment** .
8. **fold_id** is an integer, if split == 5_fold_cv_split the fold_id should be 0,1,2,3 or 4 (one less than the corresponding folder's name for this fold), in any other cases the fold_id is not used
9. **screening_threshold** only keep hits with predicted distance less than this threshold in the results.

In [12]:
from deepSIBA_screening import siba_screening
siba_screening(screening_params, model_params)

The selected chemical structure has at least one structural analogue in the training set, chembl and cmap will be screened
Finished screening for part 1/9 for the Chembl database
Finished screening for part 2/9 for the Chembl database
Finished screening for part 3/9 for the Chembl database
Finished screening for part 4/9 for the Chembl database


KeyboardInterrupt: 